In [1]:
from datasets import load_dataset, load_from_disk
import wordninja
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import re
import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, default_data_collator, get_scheduler


import torch
from torch.optim import AdamW

import collections


from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

/usr/local/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!export HF_DATASETS_CACHE="/home/"

In [3]:
#論文中の記述(4ページ), "tokenizer based on English Wikipedia unigram frequencies"
#Huggingfaceのトークナイザで上記に該当するの
    #bert-base-uncased**
    #roberta-base
    #distilbert-base-uncased
    #distilbert-base-multilingual-cased
#**実験に利用
#unigram freqencies?tokenizerのアルゴリズムは結局何なんだろう? Unigram or WordPiece?
#
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


def tokenize_function(example):
    """
    A function that tokenizes all logs
    """
    return tokenizer(example["log"], truncation=True)


# ここではログデータのロードと前処理, そしてhuggingfaceデータセットへの変換を行った

In [51]:
def log2list(path):
    with open(path, "r") as f:
        lines = [line.rstrip("\n") for line in f]
    return lines

In [61]:
df_hdfs_anomaly = pd.read_csv("./logdata/hdfs/anomaly_label.csv")
df_hdfs_anomaly.head()

,BlockId,Label
0,blk_-1608999687919862906,Normal
1,blk_7503483334202473044,Normal
2,blk_-3544583377289625738,Anomaly
3,blk_-9073992586687739851,Normal
4,blk_7854771516489510256,Normal


In [73]:
m.group(0)

'blk_-66330728533676520'

In [94]:
df_hdfs_logs.columns

Index(['Unnamed: 0', 'log', 'BlockId', 'Label'], dtype='object')

In [95]:
df_hdfs_logs = pd.read_csv("./logdata/hdfs/hdfs_log_label_pairs.csv", usecols=['log', 'BlockId', 'Label'])


In [97]:
df_hdfs_logs["label"] = df_hdfs_logs["Label"].map(lambda x : 0 if x=="Normal" else 1)

In [100]:
df_hdfs_logs = df_hdfs_logs.rename(columns={"log":"text"})
df_hdfs_logs.head()

,text,BlockId,Label,label
0,081109 203518 143 INFO dfs.DataNode$DataXceive...,blk_-1608999687919862906,Normal,0
1,081109 203518 35 INFO dfs.FSNamesystem: BLOCK*...,blk_-1608999687919862906,Normal,0
2,081109 203519 143 INFO dfs.DataNode$DataXceive...,blk_-1608999687919862906,Normal,0
3,081109 203519 145 INFO dfs.DataNode$DataXceive...,blk_-1608999687919862906,Normal,0
4,081109 203519 145 INFO dfs.DataNode$PacketResp...,blk_-1608999687919862906,Normal,0


In [101]:
def preprocess_log(text, remove_digits=True):
    text = text.replace('[', " ")
    text = text.replace(".", " ")
    text = text.replace(",", " ")
    text = text.replace(":", " ")
    text = text.replace("/", " ")
    text = text.replace(";", " ")
    text = text.replace("=", " ")
    text = text.replace("*", " ")
    text = text.replace("_", " ")
    text = text.replace("-", " ")
    text = text.lower()
    text = " ".join(wordninja.split(text))
    remove_num = lambda eg : " ".join([word for word in eg.split(" ") if not word.isdigit()])
    if remove_digits:
        return remove_num(text)
    return text;

In [85]:
# df_hdfs_logs = pd.DataFrame({"log" : hdfs})
# df_hdfs_logs["BlockId"] = df_hdfs_logs["log"].map(lambda line : re.search(r"blk_[-]*[0-9]+", line).group(0))
# df_hdfs = df_hdfs_logs.merge(df_hdfs_anomaly, how="inner", on=["BlockId"])
# df_hdfs.to_csv("./logdata/hdfs/hdfs_log_label_pairs.csv")

In [86]:
# hdfs = log2list("./logdata/hdfs/HDFS.log")
# log_label_pairs = {}
# pbar = tqdm(hdfs)
# for line in hdfs:
#     pbar.update(1)
#     m = re.search(r"blk_[-]*[0-9]+", line).group(0)
#     blkid = m.group(0)
#     log_label_pairs[line] = df_hdfs_anomaly[df_hdfs_anomaly["BlockId"] == blkid]["Label"]
# df_hdfs_logs = pd.DataFrame({"log" : hdfs})
# df_hdfs_logs["BlockId"] = df_hdfs_logs["log"].map(lambda line : re.search(r"blk_[-]*[0-9]+", line).group(0))
# df_hdfs = df_hdfs_logs.merge(df_hdfs_anomaly, how="inner", on=["BlockId"])
# df_hdfs.to_csv("./logdata/hdfs/hdfs_log_label_pairs.csv")

In [58]:
#ログデータのロード
hdfs = log2list("./logdata/hdfs/HDFS.log")
bgl = log2list("./logdata/bgl/BGL.log")
hpc = log2list("./logdata/hpc/HPC.log")
zookeeper = log2list("./logdata/zookeeper/Zookeeper.log")
proxifier = log2list("./logdata/proxifer/Proxifier.log")

hadoop = []
for path in glob.glob("./logdata/hadoop/**/*.log"):
    hadoop += log2list(path)
len(hadoop)

#NOTE
#Switchのみちょっとどうやって特徴テンプレートに埋め込んだのか分からなかったので,元論文よりデータ少なめです
switch = []
for path in glob.glob("./logdata/switch/**/template.txt"):
    switch += log2list(path)
len(switch)

959

In [191]:
print("HDFS:", len(hdfs))
print("BGL", len(bgl))
print("HPC", len(hpc))
print("Zookeeper", len(zookeeper))
print("Proxifier", len(proxifier))
print("Switch", len(switch))
print("Hadoop", len(hadoop))

HDFS: 11175629
BGL 4747963
HPC 433490
Zookeeper 74380
Proxifier 21329
Switch 959
Hadoop 394310


In [61]:
data = hdfs + bgl + hpc + zookeeper + proxifier + switch + hadoop

In [62]:
data = list(set(data))

In [165]:
target_data = data.copy()

In [99]:
data[1]

'- 1121495637 2005.07.15 R20-M0-NB-C:J08-U11 2005-07-15-23.33.57.541740 R20-M0-NB-C:J08-U11 RAS KERNEL INFO generating core.28730'

In [ ]:
target = []
#NOTE
#ここで記号を区切ってwordninjaで塊除去を行っている
#なお, かなり時間がかかるので実行には注意が必要
#もう少し賢く書きたかったが, その書き方を思いつく時間とナイーブに実行する時間はどうせ同じぐらいだろう
# for i, text in enumerate(tqdm(data)):
#     text = text.replace('[', " ")
#     text = text.replace(".", " ")
#     text = text.replace(",", " ")
#     text = text.replace(":", " ")
#     text = text.replace("/", " ")
#     text = text.replace(";", " ")
#     text = text.replace("=", " ")
#     text = text.replace("*", " ")
#     text = text.replace("_", " ")
#     text = text.replace("-", " ")
#     text = text.lower()
#     target_data[i] = " ".join(wordninja.split(text))
# target_array = np.array(target_data)
# df_log = pd.DataFrame(target_array, columns=["log"])
# df_log = df_log[df_log.log != ""]
# df_log.to_csv("./logdata/all_logs.csv")

In [ ]:
train

In [172]:
df_log

,log
1,1121495637 2005 07 15 r 20 m 0 nb c j 08 u 11 ...
2,081111 023358 19 info df s fs data set deletin...
3,081110 213754 33 info df s fs name system bloc...
4,081110 230147 16884 info df s data node data x...
5,1131981710 2005 11 14 r 44 m 0 nb c j 09 u 01 ...
...,...
16604700,081111 075637 28 info df s fs name system bloc...
16604701,081110 022458 13 info df s data block scanner ...
16604702,081110 210159 28 info df s fs name system bloc...
16604703,081111 055917 21209 info df s data node packet...


In [190]:
# #訓練データとテストデータに分ける
# TRAIN_TEST_SPLIT_RATE = 0.8
# df_log["is_train"] = np.array([np.random.rand() < TRAIN_TEST_SPLIT_RATE for _ in range(len(df_log))])
# df_log[~df_log.is_train].to_csv("./logdata/test_logs.csv")
# df_log[df_log.is_train].to_csv("./logdata/train_logs.csv")

# データセットのロード

In [49]:
dataset = load_dataset("csv", data_files = "./logdata/all_logs.csv", cache_dir="/home/")

Found cached dataset csv (/home/csv/default-a5653168cf0fdeb7/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)

100%|██████████| 1/1 [00:00<00:00,  6.00it/s]


In [4]:
dataset = dataset["train"].train_test_split(test_size=0.2)

In [5]:
# shuffled_dataset = dataset["train"].shuffle(seed=42)

In [6]:
#とりあえず変な記号などは消えた
# shuffled_dataset['log'][:10]

In [7]:
#データセットのトークナイズ
# tokenized_datasets = shuffled_train.map(tokenize_function, batched=True)

In [5]:
train_dataset = dataset["train"]

def get_training_corpus():
    for i in tqdm(range(0, len(train_dataset), 1000)):
        yield train_dataset[i : i + 1000]["log"]
        
def get_training_corpus_without_numbers():
    for i in tqdm(range(0, len(train_dataset), 1000)):
        yield train_dataset[i : i + 1000]["log_without_numbers"]
        
def get_training_corpus_small():
    for i in tqdm(range(0, len(train_dataset_small_samples), 1000)):
        yield train_dataset_small_samples[i : i + 1000]["log"]

In [9]:
num_tokens = 0
num_samples = 0
MILLION = 10**6
THRESHOLD = 100 * MILLION
for batch in get_training_corpus():
    num_tokens += sum([len(line.split(" ")) for line in batch])
    num_samples += 1000
    if num_tokens > THRESHOLD:
        break
ave_tokens = num_tokens / len(train_dataset)
print(num_tokens, ave_tokens)
LIMIT = num_samples
shuffled_dataset = train_dataset.shuffle(seed=42)
train_dataset_small_samples = shuffled_dataset.select(range(LIMIT))

 24%|██▍       | 3165/13284 [00:22<01:10, 143.63it/s]

100031984 7.530395114697544


# オリジナルのトークナイザからの訓練

In [31]:
# pre_tokenizer = pre_tokenizers.WhitespaceSplit()
# pre_tokenizer.pre_tokenize_str(train_dataset[1000]["log"])

In [104]:
# tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
# special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
# trainer = trainers.WordPieceTrainer(
#     vocab_size=5000, special_tokens=special_tokens, unk_token="<unk>"
# )
# tokenizer.train_from_iterator(get_training_corpus_small(), trainer=trainer)
# tokenizer.save(f"./log_tokenizer.json")

In [105]:
# encoding = tokenizer.encode(train_dataset[100]["log"])
# print(encoding.tokens)

# BERTトークナイザからの訓練

In [53]:
# old_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# old_tokenizer(train_dataset[1000]["log"])
# new_tokenizer = old_tokenizer.train_new_from_iterator(get_training_corpus_without_numbers() , 25000)
# new_tokenizer.save_pretrained(f"./log_tokenizer_from_old_without_numbers_large")
# new_tokenizer(train_dataset[1000]["log"])

In [3]:
MODEL_MAX_LENGTH = 180
log_tokenizer = AutoTokenizer.from_pretrained("./tokenizers/log_tokenizer_from_old_large/", model_max_length=MODEL_MAX_LENGTH)
log_tokenizer_w_n = AutoTokenizer.from_pretrained("./tokenizers/log_tokenizer_from_old_without_numbers/", model_max_length=MODEL_MAX_LENGTH, padding=True)

In [7]:
encoding = log_tokenizer(train_dataset[999]["log"])
print(encoding.tokens())

['[CLS]', '081110', '2149', '##34', '1553', '##8', 'info', 'df', 's', 'data', 'node', 'packet', 'responder', 'packet', 'responder', '2', 'for', 'block', 'b', 'lk', '7019', '##2130', '##7309', '##51', '##8527', '##0', 'terminating', '[SEP]']


In [8]:
encoding = log_tokenizer_w_n(train_dataset[999]["log_without_numbers"])
print(encoding.tokens())

['[CLS]', 'info', 'df', 's', 'data', 'node', 'packet', 'responder', 'packet', 'responder', 'for', 'block', 'b', 'lk', 'terminating', '[SEP]']


# ログから数字除去したコード

In [ ]:
# def remove_numerical(example):
#     remove_num = lambda eg : " ".join([word for word in eg.split(" ") if not word.isdigit()])
#     example["log"] = [remove_num(eg) for eg in example["log"]]
#     return example

# dataset_without_numbers = dataset.map(remove_numerical, batched=True)
# train_log_without_numbers = train_dataset_without_numbers.to_pandas()
# train_log_without_numbers.set_index(train_log_without_numbers["Unnamed: 0"])
# train_log_without_numbers = train_log_without_numbers.iloc[:, 1:]
# ds_without_numbers = dataset["train"].add_column("log_without_numbers", dataset_without_numbers["train"]["log"])
# df_without_numbers = ds_without_numbers.to_pandas()
# df_without_numbers

In [10]:
# データをトークナイズした箇所
# 時間かかるから保存済みなので, あまり実行しないで良いかな
def tokenize_function(examples):
    result = log_tokenizer_w_n(examples["log_without_numbers"])
    if log_tokenizer_w_n.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = train_dataset.map(
    tokenize_function, batched=True, remove_columns=["log", 'Unnamed: 0', 'Unnamed: 0.1', 'log_without_numbers']
)

In [13]:
chunk_size = 180

In [153]:
tokenized_samples = tokenized_datasets[:3]
for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Log {idx} length: {len(sample)}'")

'>>> Log 0 length: 16'
'>>> Log 1 length: 40'
'>>> Log 2 length: 25'


In [155]:
tokenized_samples.keys()

dict_keys(['Unnamed: 0', 'Unnamed: 0.1', 'log_without_numbers', 'input_ids', 'token_type_ids', 'attention_mask', 'word_ids'])

In [156]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in 
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated log length: {total_length}'")

'>>> Concatenated log length: 81'


In [161]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 81'


In [175]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [21]:
def add_labels_column(examples):
    result = examples
    result["labels"] = result["input_ids"].copy()
    return result

In [176]:
# 一時間かかるからなるべくやらないで
# lm_datasets = tokenized_datasets.map(group_texts, batched=True)
# lm_datasets

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 1519807
})

In [186]:
id2token = {v:k for k, v in log_tokenizer_w_n.vocab.items()}

In [189]:
log_tokenizer_w_n.decode(lm_datasets[1]["input_ids"])

'info df s fs name system block name system delete b lk is added to invalid set of [SEP] [CLS] info df s data node packet responder received block b lk of size from [SEP] [CLS] info rm communicator al locator org apache had oop map reduce v app rm rm container al locator re calculating schedule headroom memory v cores [SEP] [CLS] info df s fs name system block name system delete b lk is added to invalid set of [SEP] [CLS] info df s data node packet responder received block b lk of size from [SEP] [CLS] r m nf c j u r m nf c j u ras kernel info iar dear c c [SEP] [CLS] info df s fs data set deleting block b lk file m nt had oop df s data current sub dir b lk [SEP] [CLS] r m nf c j u r m nf c j u ras kernel info iar dear bec [SEP] [CLS] null null ras m mcs error ido proxy db hit assert condition assert expression source'

In [5]:
lm_datasets = load_from_disk("./logdata/logdata_for_pretraining/")

In [6]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=log_tokenizer_w_n, mlm_probability=0.15)
samples = [lm_datasets[i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {log_tokenizer_w_n.decode(chunk)}'")

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



'>>> [CLS] info df s data node packet responder received block b lk of size from [SEP] [CLS] r m n c [MASK] u [MASK] m n c [MASK] u ras kernel info microseconds spent in the [MASK] s signal handler during calls microseconds was the [MASK] time [MASK] a single instance [MASK] a correctable [MASK] [SEP] [CLS] info df s fs data set deleting block b lk file m nt had oop df s data current sub dir [MASK] lk [SEP] [CLS] info df s fs name system block name system add stored block block map updated is added [MASK] b [MASK] [MASK] [SEP] [CLS] [MASK] df s data node [MASK] x ce iver receiving block b lk s rc [MASK] st [SEP] [CLS] info df s fs data set deleting block b lk file m nt had oop df s data current sub dir b lk [SEP] [CLS] r m ne c j u r m ne c j u ras kernel info torus receiver z input pipe error s d cr x f detected and corrected [SEP] [CLS]'

'>>> info df s fs name system block [MASK] [MASK] delete [MASK] lk is added to invalid set of [SEP] [CLS] [MASK] df [MASK] data [MASK] packet resp

In [4]:
lm_datasets_from_disk = load_from_disk("./logdata/logdata_for_pretraining/")

In [5]:
import copy

#Unilog元論文(https://arxiv.org/pdf/2112.03159.pdf)の表2の連続マスクを作成するための関数
#ランダムでマスクするインデックスの前後もマスクするようにした
def make_consecutive_mask(mask):
    new_mask = copy.deepcopy(mask)
    
    masked_indexes = np.where(mask)[0]
    
    for i, idx in enumerate(masked_indexes):
        if idx - 1 >= 0 and idx + 1 < len(new_mask):
            if i == 0 or abs(idx - masked_indexes[i-1]) > 3:
                new_mask[idx - 1] = 1
                new_mask[idx + 1] = 1
            else:
                new_mask[idx] = 0
        else:
            new_mask[idx] = 0
    return new_mask

In [6]:


wwm_probability = 0.10

#参考https://huggingface.co/learn/nlp-course/chapter7/3?fw=pt
def consecutive_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        # CHANGED
        # 論文準拠にマスクを連続的にした
        mask = make_consecutive_mask(mask)
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = log_tokenizer_w_n.mask_token_id
        feature["labels"] = new_labels
    # print(features)
    return default_data_collator(features)

def consecutive_word_masking_data_collator_with(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        # CHANGED
        # 論文準拠にマスクを連続的にした
        mask = make_consecutive_mask(mask)
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = log_tokenizer_w_n.mask_token_id
        feature["labels"] = new_labels
    print(features)
    return default_data_collator(features)

In [7]:
samples = [lm_datasets_from_disk[i] for i in range(2)]
batch = consecutive_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {log_tokenizer_w_n.decode(chunk)}'")


'>>> [CLS] info [MASK] [MASK] [MASK] node packet responder received [MASK] [MASK] [MASK] of size from [SEP] [CLS] r m n c j u r m n c j u ras kernel info microseconds spent in the rb s signal handler during calls microseconds was the maximum [MASK] [MASK] [MASK] single instance of a correctable [MASK] [SEP] [CLS] [MASK] [MASK] s fs data set deleting block b lk file m nt had oop df s data current sub dir b [MASK] [SEP] [CLS] [MASK] [MASK] s fs [MASK] [MASK] [MASK] name system add stored block block map [MASK] [MASK] [MASK] to b lk size [SEP] [CLS] info df s data node [MASK] [MASK] ce iver receiving block b lk s rc de st [SEP] [CLS] info df s fs data set deleting block b lk file m nt had oop [MASK] [MASK] [MASK] current sub dir b lk [SEP] [CLS] r m ne c j u r m [MASK] [MASK] [MASK] u ras kernel [MASK] [MASK] [MASK] z input pipe error s d cr x f detected and corrected [SEP] [CLS]'

'>>> info df s fs name system [MASK] [MASK] [MASK] delete b lk is added to invalid set of [SEP] [CLS] [MASK

In [8]:
# tokenized_datasets = tokenized_datasets.map(add_labels_column, batched=True)

In [9]:
# samples = [tokenized_datasets[i] for i in range(2)]
# batch = consecutive_word_masking_data_collator(samples)

# for chunk in batch["input_ids"]:
#     print(f"\n'>>> {log_tokenizer_w_n.decode(chunk)}'")

In [10]:
from hf_transformers.src.transformers.models.bert.modeling_bert import BertForPreTraining
from hf_transformers.src.transformers.models.bert.configuration_bert import BertConfig
from torch.utils.data import DataLoader

# 訓練用のパラメータ設定

In [11]:
lm_datasets = lm_datasets_from_disk.train_test_split(test_size=0.1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 128

In [48]:
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1367826
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 151981
    })
})

In [12]:
lm_datasets["train"]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 1367826
})

In [13]:
train_dataloader = DataLoader(
    lm_datasets["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=consecutive_word_masking_data_collator
)

In [14]:
# train_dataset_small = lm_datasets["train"].select(range(batch_size*3))
# train_dataloader_small = DataLoader(
#     train_dataset_small,
#     shuffle=True,
#     batch_size=batch_size,
#     collate_fn=consecutive_word_masking_data_collator
# )

In [15]:
def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    masked_inputs = consecutive_word_masking_data_collator(features)
    # Create a new "masked" column for each column in the dataset
    return {"masked_" + k: v.numpy() for k, v in masked_inputs.items()}

In [16]:
eval_dataset = lm_datasets["test"].map(
    insert_random_mask,
    batched=True,
    remove_columns=lm_datasets["test"].column_names,
)
eval_dataset = eval_dataset.rename_columns(
    {
        "masked_input_ids": "input_ids",
        "masked_attention_mask": "attention_mask",
        "masked_token_type_ids": "token_type_ids",
        "masked_labels": "labels",
    }
)

In [17]:
eval_dataloader = DataLoader(
    eval_dataset, batch_size=batch_size, collate_fn=default_data_collator
)

In [16]:
# eval_dataloader_small = DataLoader(
#     eval_dataset.select(range(batch_size*3)), batch_size=batch_size, collate_fn=default_data_collator
# )

In [18]:
#Unilog元論文の実験に準拠
#https://arxiv.org/pdf/2112.03159.pdf
unilogConfig= BertConfig(
    is_unilog=True,
    attention_probs_dropout_prob=0.3,
    hidden_dropout_prob=0.3,
    num_attention_heads=4,
    hidden_size=128,
    intermediate_size=512,
    vocab_size=log_tokenizer_w_n.vocab_size,
    num_hidden_layers=3
)
unilogConfig

BertConfig {
  "attention_probs_dropout_prob": 0.3,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.3,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "is_only_mlm": false,
  "is_unilog": true,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 4,
  "num_hidden_layers": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 4075
}

In [29]:
bertConfig= BertConfig(
    attention_probs_dropout_prob=0.3,
    hidden_dropout_prob=0.3,
    num_attention_heads=4,
    hidden_size=128,
    intermediate_size=512,
    vocab_size=log_tokenizer_w_n.vocab_size,
    num_hidden_layers=12,
    is_only_mlm=True
)
bertConfig

BertConfig {
  "attention_probs_dropout_prob": 0.3,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.3,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "is_only_mlm": true,
  "is_unilog": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 4075
}

In [31]:
# del model
model = BertForPreTraining(config=bertConfig).to(device)
optimizer = AdamW(model.parameters(), lr=5e-4)

num_train_steps = 2**16 / (len(lm_datasets["train"]) // batch_size)

num_train_epochs = int(np.ceil(num_train_steps))
decayRate = 0.96
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)

In [42]:
len(lm_datasets["train"])

1367826

In [ ]:
len(lm_datasets["train"])

In [32]:
# from transformers import AutoModelForPreTraining

# checkpoint = "bert-base-uncased"
# model = AutoModelForPreTraining.from_pretrained(checkpoint, num_labels=2)


In [33]:
print(num_train_epochs)
output_dir = "./logdata/bert_log_pretrain_12_layers"
print(len(train_dataloader))

7
10687


# 事前学習開始

In [ ]:
from tqdm.auto import tqdm
import torch
import math

progress_bar = tqdm(range(num_train_epochs))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    pbar = tqdm(train_dataloader)
    for batch in tqdm(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        pbar.set_description(f">>> Loss {loss}")
        pbar.update(1)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    # Evaluation
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(loss.repeat(batch_size))

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]
    # try:
    perplexity = math.exp(torch.mean(losses))
    # except OverflowError:
    #     perplexity = float("inf")
    progress_bar.update(1)
    progress_bar.write(f">>> Epoch {epoch}: Perplexity: {perplexity}")
    # Save and upload
    model.save_pretrained(output_dir + f"_{epoch}")
    # if accelerator.is_main_process:
        # tokenizer.save_pretrained(output_dir)
        # repo.push_to_hub(
        #     commit_message=f"Training in progress epoch {epoch}", blocking=False
        # )

In [63]:
# batch["input_ids"].shape
# for chunk in batch["input_ids"]:
#     print(f"\n'>>> {log_tokenizer_w_n.decode(chunk)}'")

In [ ]:
BertForPreTraining

In [31]:
untrained_model = BertForPreTraining(config=unilogConfig).to(device)
model6 = BertForPreTraining.from_pretrained(f"./logdata/unilog_pretrain_preln_on_attentions_6/").to(device)

pbar = tqdm(train_dataloader)
for batch in pbar:
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = untrained_model(**batch)
    loss = outputs.loss
    loss.backward()
    pbar.set_description(f">>> Loss {loss}")
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    break

i = 0
# for name, p in untrained_model.named_parameters():
#     print(name)
#     print(p.grad.data)
#     i += 1
    
#     if i == 5:
#         break

pbar = tqdm(train_dataloader)
for batch in pbar:
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model6(**batch)
    loss = outputs.loss
    loss.backward()
    pbar.set_description(f">>> Loss {loss}")
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    break
i = 0
# for name, p in model6.named_parameters():
#     print(name)
    
#     print(p.grad.data)
#     i += 1
    
#     if i == 5:
#         break
        


>>> Loss 8.312609672546387:   0%|          | 0/10687 [00:00<?, ?it/s]

>>> Loss 4.698516845703125:   0%|          | 0/10687 [00:00<?, ?it/s]


In [97]:
model0 = BertForPreTraining.from_pretrained(f"./logdata/unilog_pretrain_{epoch}/").to(device)

In [98]:
for param0, param6 in zip(model0.parameters(), model6.parameters()):
    print(param0 == param6)
    break

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]], device='cuda:0')


In [55]:
import gc
# progress_bar = tqdm(range(num_train_epochs))
# model6 = BertForPreTraining.from_pretrained("./logdata/unilog_pretrain_6/").to(device)
for epoch in tqdm(range(6,3,-1)):
    model6 = BertForPreTraining.from_pretrained(f"./logdata/unilog_pretrain_preln_on_attentions_{epoch}/").to(device)
    gc.collect()
    model6.eval()
    losses = []

    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model6(**batch)

        loss = outputs.loss
        losses.append(loss.repeat(batch_size))
        # pbar.update(1)

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]
    # try:
    perplexity = math.exp(torch.mean(losses))
    # except OverflowError:
    #     perplexity = float("inf")

    progress_bar.write(f">>> Epoch {epoch}: Perplexity: {perplexity}")
    break


  0%|          | 0/3 [00:00<?, ?it/s]
                                        
  0%|          | 0/3 [00:52<?, ?it/s]/s]

>>> Epoch 6: Perplexity: 103.80807823718301


In [44]:
sample_model = BertForPreTraining.from_pretrained("./logdata/unilog_pretrain_preln_on_attentions_0/")

In [ ]:
a = loss.repeat(batch_size)
b = loss.repeat(batch_size)
torch.cat([a, b])